In [1]:
import sys
print(sys.prefix)

C:\Users\Schalk\anaconda3\envs\quinn_env


In [2]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
import os, getpass

In [3]:
from dotenv import load_dotenv
load_dotenv()

PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
PINECONE_INDEX_NAME = os.environ["PINECONE_INDEX_NAME"]

In [5]:
from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key=PINECONE_API_KEY)

In [7]:
model_name = 'text-embedding-ada-002'  
embeddings = OpenAIEmbeddings(  
    model=model_name,  
    openai_api_key=OPENAI_API_KEY
)

vectorstore = PineconeVectorStore(index_name=PINECONE_INDEX_NAME, embedding=embeddings, namespace = 'schalk-burger')
retriever = vectorstore.as_retriever()

In [8]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [9]:
# RAG prompt

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda

template = """Answer the question based only on the following context:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [10]:
chain = (
   RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
   | prompt
   | llm
   | StrOutputParser()
)

In [18]:
query = 'what roles did Schalk have at Oliver Wyman?'
chain.invoke(query)

'Schalk Burger held the role of Associate at Oliver Wyman.'